In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

stock_df = pd.read_csv('indexData.csv')

2023-12-06 21:58:04.669344: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 21:58:04.695972: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-06 21:58:04.696938: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 21:58:05.193377: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
stock_NYA_df = stock_df[stock_df['Index'] == 'NYA']
stock_NYA_df = stock_NYA_df.dropna()
stock_NYA_df['Date'] = pd.to_datetime(stock_NYA_df['Date'])
stock_NYA_df['day_difference'] = stock_NYA_df['Date'].diff().dt.days

In [3]:
data = stock_NYA_df['Adj Close'].to_numpy()
data = np.reshape(data, (len(data), 1))

In [4]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [5]:
window_size = 60

X, y = [], []

for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i-window_size:i, 0])
    y.append(scaled_data[i, 0])

In [6]:
X, y = np.array(X), np.array(y)
test_split = 0.2
split_index = int(len(y) * (1-test_split))
X_train = X[:split_index]
Y_train = y[:split_index]
X_test = X[(split_index + window_size - 1):]
Y_test = y[(split_index + window_size - 1):]

indices = np.arange(len(Y_train))
np.random.shuffle(indices)
X_train = X_train[indices]
Y_train = Y_train[indices]

In [7]:
model_1d_conv = Sequential()
model_1d_conv.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model_1d_conv.add(MaxPooling1D(pool_size=2))
model_1d_conv.add(Flatten())
model_1d_conv.add(Dense(64, activation='relu'))


model_1d_conv.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 58, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 29, 32)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 928)               0         
                                                                 
 dense (Dense)               (None, 64)                59456     
                                                                 
Total params: 59,584
Trainable params: 59,584
Non-trainable params: 0
_________________________________________________________________


2023-12-06 21:58:10.626522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-06 21:58:10.626816: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
optimizer = Adam(lr=0.001)
model_1d_conv.compile(optimizer=optimizer, loss='mse')  # Use appropriate loss for your task

In [9]:
model_1d_conv.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test), verbose = 1)

Epoch 1/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0294 - val_loss: 0.0223
Epoch 2/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0201 - val_loss: 0.0203
Epoch 3/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0201 - val_loss: 0.0204
Epoch 4/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0200 - val_loss: 0.0202
Epoch 5/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0200 - val_loss: 0.0202
Epoch 6/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0200 - val_loss: 0.0202
Epoch 7/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0172 - val_loss: 0.0141
Epoch 8/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0140 - val_loss: 0.0141
Epoch 9/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0140 - val_loss: 0.0142
Epoch 10/50
348/348 [==============================] - 1s 2ms/step - loss: 0.0124 - val_loss: 0.0121

(11109, 60)